In [25]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

In [26]:
DTYPES={
  'ID': float, 'SeriousDlqin2yrs': float, 'RevolvingUtilizationOfUnsecuredLines': float, 'age': float, 
  'NumberOfTime3059DaysPastDueNotWorse': float, 'DebtRatio': float, 'MonthlyIncome': float,
  'NumberOfOpenCreditLinesAndLoans': float, 'NumberOfTimes90DaysLate': float,
  'NumberRealEstateLoansOrLines': float, 'NumberOfTime6089DaysPastDueNotWorse': float, 'NumberOfDependents': float
}

NAMES=[
  'ID', 'SeriousDlqin2yrs', 'RevolvingUtilizationOfUnsecuredLines', 'age', 'NumberOfTime3059DaysPastDueNotWorse',
  'DebtRatio', 'MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines',
  'NumberOfTime6089DaysPastDueNotWorse', 'NumberOfDependents'
]

In [27]:
training_df = pd.read_csv('cs-training.csv', names=NAMES, dtype=DTYPES, header=0)
training_df.head(3)

,ID,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime3059DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime6089DaysPastDueNotWorse,NumberOfDependents
0,1.0,1.0,0.766127,45.0,2.0,0.802982,9120.0,13.0,0.0,6.0,0.0,2.0
1,2.0,0.0,0.957151,40.0,0.0,0.121876,2600.0,4.0,0.0,0.0,0.0,1.0
2,3.0,0.0,0.658180,38.0,1.0,0.085113,3042.0,2.0,1.0,0.0,0.0,0.0


In [28]:
test_df = pd.read_csv('cs-test.csv', names=NAMES, dtype=DTYPES, header=0)
test_df.head(3)

,ID,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime3059DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime6089DaysPastDueNotWorse,NumberOfDependents
0,1.0,NaN,0.885519,43.0,0.0,0.177513,5700.0,4.0,0.0,0.0,0.0,0.0
1,2.0,NaN,0.463295,57.0,0.0,0.527237,9141.0,15.0,0.0,4.0,0.0,2.0
2,3.0,NaN,0.043275,59.0,0.0,0.687648,5083.0,12.0,0.0,1.0,0.0,2.0


In [29]:
np.seterr(divide='ignore', invalid='ignore')

def transform_dataframe(df):

  df = df.copy()

  df['UnknownNumberOfDependents'] = df.NumberOfDependents.isnull().astype(np.int)
  df['NoDependents'] = (df.NumberOfDependents == 0).astype(np.int)
  df.NumberOfDependents.fillna(0, inplace=True)

  df['UnknownMonthlyIncome'] = df.MonthlyIncome.isnull().astype(np.int)
  df['NoIncome'] = (df.MonthlyIncome == 0).astype(np.int)
  df.MonthlyIncome.fillna(0, inplace=True)

  df['ZeroDebtRatio'] = (df.DebtRatio == 0).astype(np.int)
  df['UnknownIncomeDebtRatio'] = df.DebtRatio
  df.loc[df.UnknownMonthlyIncome == 0, 'UnknownIncomeDebtRatio'] = 0
  df.loc[df.UnknownMonthlyIncome == 1, 'DebtRatio'] = 0

  df['WeirdRevolvingUtilization'] = df.RevolvingUtilizationOfUnsecuredLines
  df.loc[~(np.log(df.WeirdRevolvingUtilization) > 3), 'WeirdRevolvingUtilization'] = 0

  df['ZeroRevolvingUtilization'] = (df.RevolvingUtilizationOfUnsecuredLines == 0).astype(np.int)
  
  df.loc[np.log(df.RevolvingUtilizationOfUnsecuredLines) > 3, 'RevolvingUtilizationOfUnsecuredLines'] = 0
  
  df['LogDebt'] = np.log(df.MonthlyIncome * df.DebtRatio)
  df.LogDebt.replace(-np.inf, 0., inplace=True)
  
  df['RevolvingLines'] = df.NumberOfOpenCreditLinesAndLoans - df.NumberRealEstateLoansOrLines
  df['HasRevolvingLoans'] = (df.RevolvingLines > 0).astype(np.int)

  df['HasRealEstateLoans'] = (df.NumberRealEstateLoansOrLines > 0).astype(np.int)
  df['HasMultipleRealEstateLoans'] = (df.NumberRealEstateLoansOrLines > 2).astype(np.int)
  
  df['EligibleSS'] = (df.age > 60).astype(np.int)
  df['DTIOver33'] = ((df.NoIncome == 0) & (df.DebtRatio > 0.33)).astype(np.int)
  df['DTIOver43'] = ((df.NoIncome == 0) & (df.DebtRatio > 0.43)).astype(np.int)
  df['DisposableIncome'] = (1 - df.DebtRatio) * df.MonthlyIncome
  df.loc[df.NoIncome == 1, 'DisposableIncome'] = 0
  
  df['RevolvingToRealEstate'] = df.RevolvingLines / (1 + df.NumberRealEstateLoansOrLines)
  
  df['NumberOfTime3059DaysPastDueNotWorseLarge'] = (df.NumberOfTime3059DaysPastDueNotWorse > 90).astype(np.int)
  df['NumberOfTime3059DaysPastDueNotWorse96'] = (df.NumberOfTime3059DaysPastDueNotWorse == 96).astype(np.int)
  df['NumberOfTime3059DaysPastDueNotWorse98'] = (df.NumberOfTime3059DaysPastDueNotWorse == 98).astype(np.int)
  df['Never3059DaysPastDueNotWorse'] = (df.NumberOfTime3059DaysPastDueNotWorse == 0).astype(np.int)
  df.loc[df.NumberOfTime3059DaysPastDueNotWorse > 90, 'NumberOfTime3059DaysPastDueNotWorse'] = 0
  
  df['NumberOfTime6089DaysPastDueNotWorseLarge'] = (df.NumberOfTime6089DaysPastDueNotWorse > 90).astype(np.int)
  df['NumberOfTime6089DaysPastDueNotWorse96'] = (df.NumberOfTime6089DaysPastDueNotWorse == 96).astype(np.int)
  df['NumberOfTime6089DaysPastDueNotWorse98'] = (df.NumberOfTime6089DaysPastDueNotWorse == 98).astype(np.int)
  df['Never6089DaysPastDueNotWorse'] = (df.NumberOfTime6089DaysPastDueNotWorse == 0).astype(np.int)
  df.loc[df.NumberOfTime6089DaysPastDueNotWorse > 90, 'NumberOfTime6089DaysPastDueNotWorse'] = 0
  
  df['NumberOfTimes90DaysLateLarge'] = (df.NumberOfTimes90DaysLate > 90).astype(np.int)
  df['NumberOfTimes90DaysLate96'] = (df.NumberOfTimes90DaysLate == 96).astype(np.int)
  df['NumberOfTimes90DaysLate98'] = (df.NumberOfTimes90DaysLate == 98).astype(np.int)
  df['Never90DaysLate'] = (df.NumberOfTimes90DaysLate == 0).astype(np.int)
  df.loc[df.NumberOfTimes90DaysLate > 90, 'NumberOfTimes90DaysLate'] = 0
  
  df['IncomeDivBy10'] = ((df.MonthlyIncome % 10) == 0).astype(np.int)
  df['IncomeDivBy100'] = ((df.MonthlyIncome % 100) == 0).astype(np.int)
  df['IncomeDivBy1000'] = ((df.MonthlyIncome % 1000) == 0).astype(np.int)
  df['IncomeDivBy5000'] = ((df.MonthlyIncome % 5000) == 0).astype(np.int)
  
  df['FullUtilization'] = (df.RevolvingUtilizationOfUnsecuredLines == 1).astype(np.int)
  df['ExcessUtilization'] = (df.RevolvingUtilizationOfUnsecuredLines > 1).astype(np.int)
  
  df['NumberOfTime3089DaysPastDueNotWorse'] = \
    df.NumberOfTime3059DaysPastDueNotWorse + df.NumberOfTime6089DaysPastDueNotWorse
    
  df['Never3089DaysPastDueNotWorse'] = df.Never3059DaysPastDueNotWorse * df.Never6089DaysPastDueNotWorse
  
  df['NumberOfTimesPastDue'] = \
    df.NumberOfTime3059DaysPastDueNotWorse + df.NumberOfTime6089DaysPastDueNotWorse + df.NumberOfTimes90DaysLate
    
  df['NeverPastDue'] = \
    df.Never3059DaysPastDueNotWorse * df.Never6089DaysPastDueNotWorse * df.Never90DaysLate
    
  df['LogRevolvingUtilizationTimesLines'] = np.log1p(df.RevolvingLines * df.RevolvingUtilizationOfUnsecuredLines)
  
  df['LogRevolvingUtilizationOfUnsecuredLines'] = np.log(df.RevolvingUtilizationOfUnsecuredLines)
  df.LogRevolvingUtilizationOfUnsecuredLines.replace(-np.inf, 0., inplace=True)
  df.drop('RevolvingUtilizationOfUnsecuredLines', axis=1, inplace=True)
  
  df['DelinquenciesPerLine'] = df.NumberOfTimesPastDue / df.NumberOfOpenCreditLinesAndLoans
  df.loc[df.NumberOfOpenCreditLinesAndLoans ==  0, 'DelinquenciesPerLine'] = 0
  
  df['MajorDelinquenciesPerLine'] = df.NumberOfTimes90DaysLate / df.NumberOfOpenCreditLinesAndLoans
  df.loc[df.NumberOfOpenCreditLinesAndLoans == 0, 'MajorDelinquenciesPerLine'] = 0
  
  df['MinorDelinquenciesPerLine'] = df.NumberOfTime3089DaysPastDueNotWorse / df.NumberOfOpenCreditLinesAndLoans
  df.loc[df.NumberOfOpenCreditLinesAndLoans == 0, 'MinorDelinquenciesPerLine'] = 0
  
  df['DelinquenciesPerRevolvingLine'] = df.NumberOfTimesPastDue / df.RevolvingLines
  df.loc[df.RevolvingLines == 0, 'DelinquenciesPerRevolvingLine'] = 0
  df['MajorDelinquenciesPerRevolvingLine'] = df.NumberOfTimes90DaysLate / df.RevolvingLines
  df.loc[df.RevolvingLines == 0, 'MajorDelinquenciesPerRevolvingLine'] = 0
  df['MinorDelinquenciesPerRevolvingLine'] = df.NumberOfTime3089DaysPastDueNotWorse / df.RevolvingLines
  df.loc[df.RevolvingLines == 0, 'MinorDelinquenciesPerRevolvingLine'] = 0
  
  df['LogDebtPerLine'] = df.LogDebt - np.log1p(df.NumberOfOpenCreditLinesAndLoans)
  df['LogDebtPerRealEstateLine'] = df.LogDebt - np.log1p(df.NumberRealEstateLoansOrLines)
  df['LogDebtPerPerson'] = df.LogDebt - np.log1p(df.NumberOfDependents)
  df['RevolvingLinesPerPerson'] = df.RevolvingLines / (1 + df.NumberOfDependents)
  df['RealEstateLoansPerPerson'] = df.NumberRealEstateLoansOrLines / (1 + df.NumberOfDependents)
  df['YearsOfAgePerDependent'] = df.age / (1 + df.NumberOfDependents)
  
  df['LogMonthlyIncome'] = np.log(df.MonthlyIncome)
  df.LogMonthlyIncome.replace(-np.inf, 0., inplace=True)
  df.drop('MonthlyIncome', axis=1, inplace=True)
  
  df['LogIncomePerPerson'] = df.LogMonthlyIncome - np.log1p(df.NumberOfDependents)
  df['LogIncomeAge'] = df.LogMonthlyIncome - np.log1p(df.age)
  
  df['LogNumberOfTimesPastDue'] = np.log(df.NumberOfTimesPastDue)
  df.LogNumberOfTimesPastDue.replace(-np.inf, 0., inplace=True)
  
  df['LogNumberOfTimes90DaysLate'] = np.log(df.NumberOfTimes90DaysLate)
  df.LogNumberOfTimes90DaysLate.replace(-np.inf, 0., inplace=True)

  df['LogNumberOfTime3059DaysPastDueNotWorse'] = np.log(df.NumberOfTime3059DaysPastDueNotWorse)
  df.LogNumberOfTime3059DaysPastDueNotWorse.replace(-np.inf, 0., inplace=True)
  
  df['LogNumberOfTime6089DaysPastDueNotWorse'] = np.log(df.NumberOfTime6089DaysPastDueNotWorse)
  df.LogNumberOfTime6089DaysPastDueNotWorse.replace(-np.inf, 0., inplace=True)

  df['LogRatio90to3059DaysLate'] = df.LogNumberOfTimes90DaysLate - df.LogNumberOfTime3059DaysPastDueNotWorse
  df['LogRatio90to6089DaysLate'] = df.LogNumberOfTimes90DaysLate - df.LogNumberOfTime6089DaysPastDueNotWorse
  
  df['AnyOpenCreditLinesOrLoans'] = (df.NumberOfOpenCreditLinesAndLoans > 0).astype(np.int)
  df['LogNumberOfOpenCreditLinesAndLoans'] = np.log(df.NumberOfOpenCreditLinesAndLoans)
  df.LogNumberOfOpenCreditLinesAndLoans.replace(-np.inf, 0., inplace=True)
  
  df['LogNumberOfOpenCreditLinesAndLoansPerPerson'] = \
    df.LogNumberOfOpenCreditLinesAndLoans - np.log1p(df.NumberOfDependents)

  df['HasDependents'] = (df.NumberOfDependents > 0).astype(np.int)
  df['LogHouseholdSize'] = np.log1p(df.NumberOfDependents)
  df.drop('NumberOfDependents', axis=1, inplace=True)

  df['LogDebtRatio'] = np.log(df.DebtRatio)
  df.LogDebtRatio.replace(-np.inf, 0., inplace=True)
  df.drop('DebtRatio', axis=1, inplace=True)

  df['LogDebtPerDelinquency'] = df.LogDebt - np.log1p(df.NumberOfTimesPastDue)
  df['LogDebtPer90DaysLate'] = df.LogDebt - np.log1p(df.NumberOfTimes90DaysLate)

  
  df['LogUnknownIncomeDebtRatio'] = np.log(df.UnknownIncomeDebtRatio)
  df.LogUnknownIncomeDebtRatio.replace(-np.inf, 0., inplace=True)
  df['LogUnknownIncomeDebtRatioPerPerson'] = df.LogUnknownIncomeDebtRatio - df.LogHouseholdSize
  df['LogUnknownIncomeDebtRatioPerLine'] = df.LogUnknownIncomeDebtRatio - np.log1p(df.NumberOfOpenCreditLinesAndLoans)
  df['LogUnknownIncomeDebtRatioPerRealEstateLine'] = \
    df.LogUnknownIncomeDebtRatio - np.log1p(df.NumberRealEstateLoansOrLines)
  df['LogUnknownIncomeDebtRatioPerDelinquency'] = df.LogUnknownIncomeDebtRatio - np.log1p(df.NumberOfTimesPastDue)
  df['LogUnknownIncomeDebtRatioPer90DaysLate'] = df.LogUnknownIncomeDebtRatio - np.log1p(df.NumberOfTimes90DaysLate)

  df['LogNumberRealEstateLoansOrLines'] = np.log(df.NumberRealEstateLoansOrLines)
  df.LogNumberRealEstateLoansOrLines.replace(-np.inf, 0., inplace=True)
  df.drop('LogNumberRealEstateLoansOrLines', axis=1, inplace=True)
  
  df.drop(
    ['NumberOfOpenCreditLinesAndLoans', 'NumberOfTimesPastDue', 'NumberOfTimes90DaysLate',
    'NumberOfTime3059DaysPastDueNotWorse', 'NumberOfTime6089DaysPastDueNotWorse'],
  axis=1, inplace=True)

  df['LowAge'] = (df.age < 18).astype(np.int)
  df['LogAge'] = np.log(df.age - 17)
  df.LogAge.replace(np.nan, 0., inplace=True)
  df.drop('age', axis=1, inplace=True)
  
  return df.astype(np.float32)

In [30]:
training_features = transform_dataframe(training_df)
training_features.to_csv('training_features.csv', index=False)

In [31]:
test_features = transform_dataframe(test_df)
test_features.to_csv('test_features.csv', index=False)